In [30]:
import pyproj
import math
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import ORBSLAMResults, umeyama_alignment, kabsch_umeyama

results = ORBSLAMResults("~/msc/shared_data/orbslam-out-merc-umeyama-vabadusepst")

gps_trajectory_wgs = pd.DataFrame([(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                              for kf in results.keyframes[1:]], columns=['lat', 'lon', 'alt'])
slam_trajectory = pd.DataFrame([(kf.x, kf.y, kf.z) for kf in results.keyframes[1:]], columns=[ 'x', 'y', 'z'])

# Create transformers for WGS84 <-> UTM35N
utm2wgs = pyproj.Transformer.from_crs(4326, 32635)
utm2wgs = pyproj.Transformer.from_crs(32635, 4326)

# Convert GPS trajectory (WGS84) to UTM35N
gps_trajectory_utm = pd.DataFrame([utm2wgs.transform(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                                   for kf in results.keyframes[1:]], columns=['x', 'y', 'z'])
# Find transformation between GPS and SLAM trajectories
R, t, c = umeyama_alignment(slam_trajectory.to_numpy().T, gps_trajectory_utm.to_numpy().T, True)

# Apply transformation to SLAM trajectory
adjusted_slam_trajectory_utm = np.array([t + c * R @ p for p in slam_trajectory.to_numpy()])
# Convert SLAM trajectory (UTM35N) to WGS84
adjusted_slam_trajectory_wgs = [utm2wgs.transform(p[0], p[1], p[2]) for p in adjusted_slam_trajectory_utm]

# Convert SLAM estimate (UTM35N) to WGS84
slam_estimate_wgs = [utm2wgs.transform(p.lat, p.lon, p.alt) for p in results.gps_estimate]

fig = px.scatter_mapbox(gps_trajectory_wgs, lat="lat", lon="lon")
fig.add_scattermapbox(lat=[p[0] for p in adjusted_slam_trajectory_wgs], lon=[p[1] for p in adjusted_slam_trajectory_wgs])
fig.add_scattermapbox(lat=[p[0] for p in slam_estimate_wgs], lon=[p[1] for p in slam_estimate_wgs])
fig.update_layout(mapbox_style="open-street-map", margin={"t": 0, "b": 0, "l": 0, "r": 0}, height=800)
fig.update_geos(projection_type="transverse mercator")
fig.show()

